In [7]:
from datasets import load_dataset

In [8]:
import spacy

In [9]:
ds = load_dataset("argilla/medical-domain")
train_data = ds['train']

In [10]:
def inspect_data(data, num_samples=5):
    # Print out the full structure of dataset
    for i, item in enumerate(data[:num_samples], 1):
        print(f"Features {i}: {item}\n")

    # Print out the structure and content of the first few data entries
    for i in range(num_samples):
        sample = data[i]
        print(f"Sample {i+1}: {sample}\n")

# Run the inspection of data
inspect_data(train_data)

Features 1: text

Features 2: inputs

Features 3: prediction

Features 4: prediction_agent

Features 5: annotation

Features 6: annotation_agent

Features 7: multi_label

Features 8: explanation

Features 9: id

Features 10: metadata

Features 11: status

Features 12: event_timestamp

Features 13: metrics

Sample 1: {'text': 'PREOPERATIVE DIAGNOSIS:,  Iron deficiency anemia.,POSTOPERATIVE DIAGNOSIS:,  Diverticulosis.,PROCEDURE:,  Colonoscopy.,MEDICATIONS: , MAC.,PROCEDURE: , The Olympus pediatric variable colonoscope was introduced into the rectum and advanced carefully through the colon to the cecum identified by the ileocecal valve and the appendiceal orifice.  Preparation was good, although there was some residual material in the cecum that was difficult to clear completely.  The mucosa was normal throughout the colon.  No polyps or other lesions were identified, and no blood was noted.  Some diverticula were seen of the sigmoid colon with no luminal narrowing or evidence of inflamm

## Standard and potential new NER types
- Standard NER types that are prominent in our data are Persons, dates, times, locations, organizations (ex. department)
- Potential new NER types that are prominent in our data are diagnosis, symptoms, treatment, and lab values. 

## Investigate LLM-based NER
### Utilizing Chatbots

In [13]:
train_data_trunc = train_data.select(range(10))

In [16]:
import subprocess
import os
import csv

def query_model(prompt, model, size):
    # Run the Ollama command in the terminal
    result = subprocess.run(
        ["ollama", "run", f"{model}:{size}"],
        input=prompt,
        text=True,
        capture_output=True
    )
    return result.stdout

def process_all_text_entries(data, model, size, csv_writer):
    for idx, item in enumerate(data):
        document_text = item['text']  # Extract the 'text' field from each entry
        print(document_text[:200])
        
        # Updated prompt to extract all NER types
        prompt = f"You are a helpful assistant tasked with extracting all named entities from the following text. For each entity, create a numbered list specifying the type (e.g., PERSON, ORGANIZATION, LOCATION, DATE, DIAGNOSIS, TREATMENT, etc.): {document_text}. Respond ONLY with this list of entities and their types."
        response = query_model(prompt=prompt, model=model, size=size)
        
        # Print response for each entry (optional)
        print(f"Response for entry {idx} using {model} {size}:\n{response}\n")
        
        # Write the response to the CSV file
        csv_writer.writerow([model, size, document_text[:200] + '...', response])  # Include only part of the text for brevity


# Create the CSV file and add headers
output_csv_path = "model_responses.csv"
with open(output_csv_path, mode="w", newline="") as csv_file:
    csv_writer = csv.writer(csv_file)
    # Write the header
    csv_writer.writerow(["Model Name", "Model Size", "Text Sample", "Response to Prompt"])

    # Call the function for each model and size configuration
    process_all_text_entries(train_data_trunc, "llama3.2", "3b", csv_writer)
    #process_all_text_entries(train_data, "llama3.1", "8b", csv_writer)

print(f"Responses have been saved to {output_csv_path}")

PREOPERATIVE DIAGNOSIS:,  Iron deficiency anemia.,POSTOPERATIVE DIAGNOSIS:,  Diverticulosis.,PROCEDURE:,  Colonoscopy.,MEDICATIONS: , MAC.,PROCEDURE: , The Olympus pediatric variable colonoscope was i
Response for entry 0 using llama3.2 3b:
1. Iron deficiency anemia - DIAGNOSIS
2. Diverticulosis - DIAGNOSIS
3. MAC - MEDICATIONS
4. Olympus pediatric variable colonoscope - ORGANIZATION
5. Dr. X - PERSON



CLINICAL INDICATION:  ,Normal stress test.,PROCEDURES PERFORMED:,1.  Left heart cath.,2.  Selective coronary angiography.,3.  LV gram.,4.  Right femoral arteriogram.,5.  Mynx closure device.,PROCEDURE
Response for entry 1 using llama3.2 3b:
1. Left heart cath (PROCEDURE)
2. Selective coronary angiography (PROCEDURE)
3. LV gram (PROCEDURE)
4. Right femoral arteriogram (PROCEDURE)
5. Mynx closure device (PROCEDURE)
6. Lidocaine (SUBSTANCE)
7. 2% lidocaine (SUBSTANCE)
8. LVEDP (MEASUREMENT)
9. Left main (ANATOMY)
10. LAD (ANATOMY)
11. Left circumflex (ANATOMY)
12. Right coronary artery (A

#### *Conclusions*

- It is relatively simple to run LLama3.2 locally. However, it needs GPUs to run efficiently
- Because Llama is designed to take an input prompt and generate text in response, it is suitable for conversational AI, interactive querying, and generating outputs based on a range of instructions
- This allowed the model to identify NER that goes beyond SpaCy's capabilities, identifying medicines, body parts, medical procedures, etc.
- However, each output was in different formats. For ideal results, careful prompt engineering and post processing would be necessary.
- After adjustments to the prompt (giving the AI a role, asking for a specific format, limiting the response to just the specified format and no additional words) the responses improved immensely

### Applying NER Models

In [29]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Load the model and tokenizer
model_name = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

### Possible models to choose from
### Universal-NER/UniNER-7B-type-sup
### emilyalsentzer/Bio_ClinicalBERT
### bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12

Loading checkpoint shards: 100%|██████████| 3/3 [00:00<00:00,  6.74it/s]
Some weights of LlamaForTokenClassification were not initialized from the model checkpoint at Universal-NER/UniNER-7B-type-sup and are newly initialized: ['score.bias', 'score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
# Create the NER pipeline
nlp_ner = pipeline("ner", model=model, tokenizer=tokenizer) # runs the specific model in NER mode

# Example text
text = "John Doe was diagnosed with Type 2 Diabetes on 2024-11-10."

# Perform NER
entities = nlp_ner(text)

# Print out the recognized entities
for ent in entities:
    print(f"Entity: {ent['word']}, Label: {ent['entity']}, Score: {ent['score']:.4f}")

Entity: ▁John, Label: LABEL_1, Score: 0.9788
Entity: ▁Do, Label: LABEL_1, Score: 0.8528
Entity: e, Label: LABEL_1, Score: 0.9956
Entity: ▁was, Label: LABEL_0, Score: 0.8023
Entity: ▁diagn, Label: LABEL_0, Score: 0.6978
Entity: osed, Label: LABEL_0, Score: 0.9122
Entity: ▁with, Label: LABEL_0, Score: 0.5410
Entity: ▁Type, Label: LABEL_1, Score: 0.5749
Entity: ▁, Label: LABEL_1, Score: 0.8181
Entity: 2, Label: LABEL_1, Score: 0.9712
Entity: ▁Di, Label: LABEL_1, Score: 0.9771
Entity: ab, Label: LABEL_1, Score: 0.9787
Entity: etes, Label: LABEL_1, Score: 0.7363
Entity: ▁on, Label: LABEL_0, Score: 0.9951
Entity: ▁, Label: LABEL_0, Score: 0.9337
Entity: 2, Label: LABEL_0, Score: 0.7496
Entity: 0, Label: LABEL_0, Score: 0.6739
Entity: 2, Label: LABEL_0, Score: 0.6688
Entity: 4, Label: LABEL_0, Score: 0.9242
Entity: -, Label: LABEL_0, Score: 0.6383
Entity: 1, Label: LABEL_0, Score: 0.5003
Entity: 1, Label: LABEL_0, Score: 0.8396
Entity: -, Label: LABEL_0, Score: 0.7545
Entity: 1, Label: LABEL_

# To do next

Fine-tune the Model on a Custom Dataset
If the predefined labels of the model (dslim/bert-base-NER) don't include DIAGNOSIS or other desired labels, you need to fine-tune it on a dataset that includes these entities.

Prepare a Custom Dataset:

Annotate text data with your desired entity labels (PERSON, DIAGNOSIS, DATE, etc.).
Format it in a standard format like CoNLL or JSONL (depending on the tokenizer and training framework).
Fine-tune the Model:

Use the transformers library to fine-tune the model on your annotated dataset.
Adjust the num_labels argument during model definition to match your label count.
Example guide for fine-tuning: Hugging Face Fine-Tuning Tutorial.

#### *Conclusions*

- Loading the model took the longest in this process
- This model was trained for the task of extracting NER from text, and was on the clinical NER leaderboard on Hugging Face
- The outputs on this test statement show that the entities that were extracted are not specific to the medical field, meaning that fine tuning would be needed to use this on our data